# Part1

In [2]:
pip install nltk

In [4]:
!pip install torch torchvision torchaudio

In [5]:
import nltk


In [6]:
import torch
print(torch.__version__)

2.3.1+cu121


In [7]:
print(torch.cuda.is_available())

False


In [8]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()



In [9]:
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    # Stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # Initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1
    return bag


# Part2


In [11]:
import json

In [12]:
with open('/bin/intents.json','r') as f:
  intents = json.load(f)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hello', 'Hi', 'How are you', 'Is anyone there?', 'Good day', 'Hey there'], 'responses': ["Hello! I'm here to listen. How are you feeling today?", "Hi! I'm here to help. What’s on your mind?", 'Hi there! How can I assist you with your thoughts and feelings today?', 'Hello! Feel free to share what’s bothering you.']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Talk to you later'], 'responses': ["Goodbye! Take care and remember, I'm here if you need to talk.", "See you later. Don't hesitate to reach out again if you need support.", 'Farewell! Wishing you peace and comfort. Come back whenever you need.', 'Talk to you later. Remember, you’re not alone.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', 'That’s helpful', 'Thanks a lot!'], 'responses': ["You're welcome! I'm here to help whenever you need.", 'Anytime! Feel free to reach out if you need support.', 'My pleasure. Remember, talking can be very helpful.'

In [13]:
all_words = []
tags = []
data_pairs = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        tokens = tokenize(pattern)
        # Add to our words list
        all_words.extend(tokens)
        # Add to data pairs
        data_pairs.append((tokens, tag))

# Stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# Remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(data_pairs), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

46 patterns
11 tags: ['coping_strategies', 'daily_motivation', 'emotional_support', 'goodbye', 'greeting', 'relationship_issues', 'seeking_help', 'self_care', 'stress_management', 'suicidal_thoughts', 'thanks']
99 unique stemmed words: ["'m", 'a', 'about', 'and', 'anxieti', 'anxiou', 'anymor', 'anyon', 'are', 'argu', 'better', 'bye', 'can', 'care', 'commit', 'consid', 'cope', 'counselor', 'day', 'depress', 'difficult', 'distant', 'do', 'emot', 'end', 'feel', 'find', 'for', 'from', 'give', 'good', 'goodby', 'handl', 'hard', 'have', 'health', 'hello', 'help', 'hey', 'hi', 'hopeless', 'how', 'i', 'in', 'is', 'issu', 'later', 'life', 'live', 'lot', 'love', 'm', 'manag', 'me', 'mental', 'motiv', 'my', 'myself', "n't", 'need', 'of', 'one', 'overwhelm', 'partner', 'posit', 'practic', 'problem', 'profession', 'quot', 'reduc', 'relationship', 's', 'see', 'self-car', 'should', 'some', 'stay', 'strategi', 'stress', 'suggest', 'suicid', 'take', 'talk', 'techniqu', 'thank', 'that', 'therapist', 'th

In [14]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in data_pairs:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)
X_train = np.array(X_train)
y_train = np.array(y_train)

In [15]:
# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

In [16]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [17]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)


# Pytorch model and training


In [18]:
import torch
import torch.nn as nn

In [19]:
# Define model
class ChatModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(ChatModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, num_labels)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        x = self.activation(x)
        x = self.layer3(x)
        return x

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ChatModel(input_dim=input_size, hidden_dim=hidden_size, num_labels=output_size).to(device)

# Loss and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (batch_words, batch_labels) in train_loader:
        batch_words = batch_words.to(device)
        batch_labels = batch_labels.to(dtype=torch.long).to(device)

        # Forward pass
        predictions = model(batch_words)
        loss = loss_function(predictions, batch_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'Final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.0609
Epoch [100/1000], Loss: 0.9443
Epoch [100/1000], Loss: 0.4393
Epoch [100/1000], Loss: 0.9156
Epoch [100/1000], Loss: 0.9391
Epoch [100/1000], Loss: 0.9562
Epoch [200/1000], Loss: 0.0866
Epoch [200/1000], Loss: 0.1252
Epoch [200/1000], Loss: 0.0557
Epoch [200/1000], Loss: 0.0309
Epoch [200/1000], Loss: 0.0767
Epoch [200/1000], Loss: 0.0855
Epoch [300/1000], Loss: 0.0100
Epoch [300/1000], Loss: 0.0164
Epoch [300/1000], Loss: 0.0163
Epoch [300/1000], Loss: 0.0137
Epoch [300/1000], Loss: 0.0194
Epoch [300/1000], Loss: 0.0122
Epoch [400/1000], Loss: 0.0085
Epoch [400/1000], Loss: 0.0056
Epoch [400/1000], Loss: 0.0057
Epoch [400/1000], Loss: 0.0060
Epoch [400/1000], Loss: 0.0046
Epoch [400/1000], Loss: 0.0025
Epoch [500/1000], Loss: 0.0027
Epoch [500/1000], Loss: 0.0035
Epoch [500/1000], Loss: 0.0034
Epoch [500/1000], Loss: 0.0021
Epoch [500/1000], Loss: 0.0022
Epoch [500/1000], Loss: 0.0027
Epoch [600/1000], Loss: 0.0017
Epoch [600/1000], Loss: 0.0014
Epoch [6

# Save and load the model

In [21]:
# Save model data
model_data = {
    "model_params": model.state_dict(),
    "input_dim": input_size,
    "hidden_dim": hidden_size,
    "num_labels": output_size,
    "word_list": all_words,
    "label_tags": tags
}

SAVE_PATH = "model_data.pth"
torch.save(model_data, SAVE_PATH)
print(f'Training complete. File saved to {SAVE_PATH}')

Training complete. File saved to model_data.pth


In [22]:
import random
import json

import torch


In [23]:
# Load the model
import random
import json

model_data = torch.load(SAVE_PATH)

input_dim = model_data["input_dim"]
hidden_dim = model_data["hidden_dim"]
num_labels = model_data["num_labels"]
word_list = model_data['word_list']
label_tags = model_data['label_tags']
model_params = model_data["model_params"]

model = ChatModel(input_dim, hidden_dim, num_labels).to(device)
model.load_state_dict(model_params)
model.eval()

ChatModel(
  (layer1): Linear(in_features=99, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (layer3): Linear(in_features=8, out_features=11, bias=True)
  (activation): ReLU()
)

In [24]:
bot_name = "Imane"
print("Hi there! (type 'quit' to exit)")
while True:
    user_input = input("User: ")
    if user_input.lower() == "quit":
        break

    tokens = tokenize(user_input)
    input_vector = bag_of_words(tokens, word_list)
    input_vector = input_vector.reshape(1, input_vector.shape[0])
    input_vector = torch.from_numpy(input_vector).to(device)

    output = model(input_vector)
    _, predicted_label = torch.max(output, dim=1)

    tag = label_tags[predicted_label.item()]

    probabilities = torch.softmax(output, dim=1)
    probability = probabilities[0][predicted_label.item()]
    if probability.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I did not get what you mean, can you reformulate?")

Hi there! (type 'quit' to exit)
User: hi
Imane: Hi there! How can I assist you with your thoughts and feelings today?
User: i am stressed
Imane: I did not get what you mean, can you reformulate?
User: i am anxious
Imane: I'm here to listen. What’s been on your mind lately?
User: i don't want to live anymore
Imane: There are people who care and want to help you. Please reach out to a crisis support service or a mental health professional.
User: and i am having problems in my relationship
Imane: Relationships can be challenging. It might help to talk about what’s been happening and how it’s affecting you.
User: What strategies can help me feel better?
Imane: Practicing relaxation techniques and finding activities that you enjoy can also be beneficial.
User: can you Help me stay positive
Imane: Staying positive can be challenging, but setting small, achievable goals can help.
User: Goodbye! Take care and remember, I'm here if you need to talk.
Imane: It's important to set aside time for y